# SVM Model Tuning

In this notebook, we will take the folds generated by the OU class in the previous notebook  and try to find the best set of parameters for our SVM to perform binary classification. 

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV

import pickle

import OU


%load_ext autoreload
%autoreload 2

In [2]:
save_dir = './data/'

In [3]:
info = np.load(save_dir + "/info.npy", allow_pickle=True)

First we need to recompile the fold dictionary into one large dataframe so that sci-kit learn's GridSearchCV class can search for the optimal parameters efficiently. 

In [4]:
splits = []
multi_cv_df = pd.DataFrame()
multi_cv_labels = pd.Series()

for i in range(len(info)):
    train = info[i]['train']['df_scale'].copy()
    train_labels = info[i]['train']['labels'].copy()
    
    test = info[i]['test']['df_scale'].copy()
    test_labels = info[i]['test']['labels'].copy()

    train_len = train.shape[0]
    test_len = test.shape[0]
    
    # Append rows to dataframe
    multi_cv_df = multi_cv_df.append(train, ignore_index=True)
    multi_cv_labels = multi_cv_labels.append(train_labels, ignore_index=True)
    
    # Append labels to a dataframe
    multi_cv_df = multi_cv_df.append(test, ignore_index=True)
    multi_cv_labels = multi_cv_labels.append(test_labels, ignore_index=True)
    
    # Append the indices of the folds to a list
    splits.append((multi_cv_df.iloc[-train_len-test_len:-test_len].index, multi_cv_df.iloc[-test_len:].index))
    
    # Quality Assurance
    assert(np.array_equal(multi_cv_df.loc[splits[i][0]].values, train.values))
    assert(np.array_equal(multi_cv_labels.loc[splits[i][0]].values, train_labels.values))
    assert(np.array_equal(multi_cv_df.loc[splits[i][1]], test.values))
    assert(np.array_equal(multi_cv_labels.loc[splits[i][1]], test_labels))
    
splits = np.array(splits)

np.save(save_dir + 'splits.npy', splits)

/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
# Save off data
multi_cv_df.to_csv(save_dir + 'df.csv')
multi_cv_labels.to_csv(save_dir + 'labels.csv')

# Gridsearch

We want to find the optimal hyperparameters for our SVM by exploring all combinations of possible hyperparameter


In [14]:
# Too much for local, use the smaller gridsearch first
# params = [{ 'kernel': ['rbf'],
#             'C': [0.1,1,10,100], 
#             'gamma': [1, 0.1, 0.001, 0.0001], 
#             'cache_size': [2000], 
#             'class_weight': [{0: 0.5, 1: 0.5}, {0: 0.6, 1: 0.4}, 
#                              {0: 0.7, 1: 0.3}, {0: 0.8, 1: 0.2}]
#           }, 
#           { 'kernel': ['poly'], 
#             'C': [0.1, 1,10,100,], 
#             'gamma': [1, 0.1, 0.001, 0.0001],
#             'degree': [3, 5],
#             'cache_size': [2000],
#             'class_weight': [{0: 0.5, 1: 0.5}, 
#                              {0: 0.6, 1: 0.4}, {0: 0.7, 1: 0.3}]
#           }]

params = [{ 'kernel': ['rbf'],
            'C': [1,10,100], 
            'gamma': [1, 0.1, 0.001], 
            'cache_size': [2000], 
            'class_weight': [{0: 0.5, 1: 0.5}, {0: 0.6, 1: 0.4}, 
                             {0: 0.7, 1: 0.3}, {0: 0.8, 1: 0.2}]
          }
          ]

In [15]:
# Use all cores (n_jobs-1)
gridcv = GridSearchCV(svm.SVC(), params, verbose=1, cv=list(splits), n_jobs=-1, 
                    scoring=['precision'], refit=False)

gridcv.fit(multi_cv_df, multi_cv_labels)

Fitting 399 folds for each of 36 candidates, totalling 14364 fits


/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xijiaqi/opt/anaconda3/envs/hft/lib/python3

GridSearchCV(cv=[array([RangeIndex(start=0, stop=2000, step=1),
       RangeIndex(start=2000, stop=2100, step=1)], dtype=object),
                 array([RangeIndex(start=2100, stop=4100, step=1),
       RangeIndex(start=4100, stop=4200, step=1)], dtype=object),
                 array([RangeIndex(start=4200, stop=6200, step=1),
       RangeIndex(start=6200, stop=6300, step=1)], dtype=object),
                 array([RangeIndex(start=6300, s...
       RangeIndex(start=60800, stop=60900, step=1)], dtype=object),
                 array([RangeIndex(start=60900, stop=62900, step=1),
       RangeIndex(start=62900, stop=63000, step=1)], dtype=object), ...],
             estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [1, 10, 100], 'cache_size': [2000],
                          'class_weight': [{0: 0.5, 1: 0.5}, {0: 0.6, 1: 0.4},
                                           {0: 0.7, 1: 0.3}, {0: 0.8, 1: 0.2}],
                          'gamma': [1, 0.1, 0.001], 'kernel': ['rbf']}],
 

In [25]:
gridcv_results = pd.DataFrame(gridcv.cv_results_)

In [32]:
best_param = gridcv_results[gridcv_results.rank_test_precision == 1]['params'].values[0]
print(f'Best SVM parameters after Grid Search is {best_param}.')

Best SVM parameters after Grid Search is {'C': 100, 'cache_size': 2000, 'class_weight': {0: 0.6, 1: 0.4}, 'gamma': 1, 'kernel': 'rbf'}.


In [24]:
# save the object to a file
with open(save_dir+'gridsearch_results.pkl', 'wb') as f:
    pickle.dump(gridcv, f)